In [1]:
import json
import csv,re,os,sys,random
from IPython.display import IFrame

In [2]:
# Parameters for ngram time series
start=u'1920'
end=u'2010'

In [3]:
with open('ontology.json','r') as inFile:
    d=json.loads(inFile.read())

In [4]:
header="""<head>\
<meta charset="UTF-8">\
</head>\
<style type="text/css">\n.iframe {\
  margin-top: 20px;\
  margin-bottom: 30px;
  padding-left: -90px;\n\
  .h1 {\
  font-size:48px;\
  align:center;\
}\
}</style>\n"""

In [5]:
termsFile=csv.reader(open('out.csv','r'),delimiter='\t')
termsDict={}

for line in termsFile:
    termsDict[line[2]]=line[0:2]
    termsDict[line[2]].extend(line[3:]+[line[2]])
    # Dictionary of each term in thesaurus, Key is English term, value is list of translations [AR,CN,FR,RU,FR,EN]
    # e.g. termsDict['electronic publications']=['لمنشورات الإلكترونية','电子出版物','publications electroniques'
    # 'Электронные издания','publicaciones electronicas','electronic publications']


In [6]:
d.keys()

[u'RU', u'FR', u'EN', u'CN', u'AR', u'ES']

In [7]:
for lang in d.keys():
    print lang,len(d[lang].keys())

RU 7107
FR 7116
EN 7118
CN 7055
AR 7082
ES 7117


In [8]:
srcString='"https://books.google.com/ngrams/interactive_chart?content=peace%2Cwar&year_start=1800&year_end=2000&corpus=18&smoothing=0&share=&direct_url=t1%3B%2Cpeace%3B%2Cc0%3B.t1%3B%2Cwar%3B%2Cc0"'

In [101]:
def getIframeSource(terms,nLang=u'EN',plotRelated=True):
    corpusHash={u'CN':23,u'FR':19,u'EN':15,u'RU':25,u'ES':21}
    # These are the IDs for each language in n-gram widget
    
    corpusId=corpusHash[nLang]
    corpusId=unicode(corpusId)
    terms=[term.decode('utf-8').lower() for term in terms]
    
    height=u'150'
    width=u'300'
    
    smoothing='10'
    
    if len(terms)==1 or (not plotRelated):
        print 'Single',lang
        returnString=u'<iframe name="%s" src="https://books.google.com/ngrams/interactive_chart?content=' % lang
        returnString+=terms[0]
        returnString+=u'&year_start=%s&year_end=%s' % (start,end)
        returnString+=u'&case_insensitive=on&corpus='
        returnString+=corpusId
        returnString+=u'&smoothing=%s&share=&direct_url=t1%%3B%%2C' % smoothing
        returnString+=terms[0]
        returnString+=u'%%3B%%2Cc0" width=%s height=%s marginwidth=0 marginheight=0 hspace=-100 vspace=0\
        frameborder=0 scrolling=no></iframe>' % (width,height)
        return returnString
    else:
        print 'Multiple',lang
        print terms
        if len(terms)>4:
            terms=terms[0:4]
            print 'Limiting:',terms
        returnString=u'<iframe name="%s" src="https://books.google.com/ngrams/interactive_chart?content=' % lang
        returnString+=u','.join(terms)
        returnString+=u'&year_start=%s&year_end=%s' % (start,end)
        returnString+=u'&case_insensitive=on&corpus='
        returnString+=corpusId
        returnString+=u'&smoothing=%s&share=&direct_url=' % smoothing
        returnString+=u'.'.join(['t1%3B%2C'+t+'%3B%2Cc0%3B' for t in terms[0:-1]])
        returnString+=u'.t1%3B%2C'+terms[-1]
        returnString+=u'%%3B%%2Cc0" width=%s height=%s \
        marginwidth=0 marginheight=0 hspace=-100 vspace=0 frameborder=0 scrolling=no></iframe>' % (width,height)
        return returnString

In [102]:
def writeLang(l):return None#htmlFile.write('<h1 align="center">%s</h1>' % l)

In [103]:
def getTermString(term):
    returnString=term+''
    return '%s (%s - %s)' % (term,start,end)

In [108]:
with open('iframes.html','w') as htmlFile:
    htmlFile.write(header)
    
    nCharts=3
    plotRelatedTerms=False
    
    for term in random.sample(d['EN'].keys(),nCharts):
        print term
        writeLang('EN')
        htmlFile.write("""<table border="0" cellpadding="10" cellspacing="0" width="100%" height="200px"><tr>\n""")
        # Open table
        termString=getTermString(term)
        htmlFile.write('<tr><th colspan="5"><h1 align="center">%s</h1></th><tr>' % termString)
        htmlFile.write("<tr><td>English (%s)</td><td>Chinese (%s)</td><td>French (%s)</td>\
        <td>Russian (%s)</td><td>Spanish (%s)</td></tr>" % tuple([term.encode('utf-8')]+(termsDict[term][1:-1])))
        # Write row of languages
        htmlFile.write('\n'+"<tr>")
        # Open row

        try:relatedTerms=d['EN'][term]['Related terms']
        except:relatedTerms=[]
            
        srcString=getIframeSource([term]+relatedTerms)
        htmlFile.write("""<div style="padding:-20px;"><td width=300px>""")
        # Open cell
        
        htmlFile.write(srcString+'\n')
        htmlFile.write("</td></div>"+'\n')
        # Close cell

        if True:
        
            for term,lang in zip(termsDict[term][1:-1],['CN','FR','RU','ES']):

                try:relatedTerms=d[lang][term]['Related terms']
                except:relatedTerms=[]

                srcString=getIframeSource([term]+relatedTerms,lang)
                htmlFile.write("<td>")
                # Open cell
                writeLang(lang)
                htmlFile.write(srcString.encode('utf-8')+'\n')
                htmlFile.write("</td><p></p>")
                # Close cell

        htmlFile.write("<tr>")
        # Close row

glossaries
Multiple ES
[u'glossaries', u'definition', u'dictionaries', u'new words', u'terminology', u'thesauri']
Limiting: [u'glossaries', u'definition', u'dictionaries', u'new words']
Single CN
Multiple FR
[u'glossaires', u'dictionnaires', u'neologismes', u'terminologie', u'thesauri']
Limiting: [u'glossaires', u'dictionnaires', u'neologismes', u'terminologie']
Single RU
Multiple ES
[u'glosarios', u'diccionarios', u'neologismos', u'terminologia', u'tesauros']
Limiting: [u'glosarios', u'diccionarios', u'neologismos', u'terminologia']
kenyans
Multiple ES
[u'kenyans', u'kenya']
Single CN
Multiple FR
[u'kenyens', u'kenya']
Single RU
Multiple ES
[u'kenianos', u'kenya']
ethiopia
Multiple ES
[u'ethiopia', u'ethiopian refugees', u'ethiopians', u'northeast africa']
Single CN
Multiple FR
[u'ethiopie', u'afrique du nord-est', u'ethiopiens', u'refugies ethiopiens']
Single RU
Multiple ES
[u'etiopia', u'africa nororiental', u'etiopes', u'refugiados etiopes']


In [97]:
d['EN'].items()[0:2]

[(u'differential equations', {u'Related terms': [u'CALCULUS']}),
 (u'jihad', {u'Related terms': [u'ISLAM', u'WAR']})]

In [86]:
"""<iframe name="ngram_chart" src="https://books.google.com/ngrams/interactive_chart?content=war%2Cpeace&year_start=1800&year_end=2000&corpus=17&smoothing=0&share=&direct_url=t1%3B%2Cwar%3B%2Cc0%3B.t1%3B%2Cpeace%3B%2Cc0" width=900 height=500 marginwidth=0 marginheight=0 hspace=0 vspace=0 frameborder=0 scrolling=no></iframe>"""

'<iframe name="ngram_chart" src="https://books.google.com/ngrams/interactive_chart?content=war%2Cpeace&year_start=1800&year_end=2000&corpus=17&smoothing=0&share=&direct_url=t1%3B%2Cwar%3B%2Cc0%3B.t1%3B%2Cpeace%3B%2Cc0" width=900 height=500 marginwidth=0 marginheight=0 hspace=0 vspace=0 frameborder=0 scrolling=no></iframe>'

In [78]:
term='peace'
srcString="https://books.google.com/ngrams/interactive_chart?content=peace,war&year_start=1800&year_end=2000&corpus=18&smoothing=0&share=&direct_url=t1%3B%2C"+term+"%3B%2Cc0"
IFrame(srcString,width=900,height=500)